In [1]:
%load_ext autoreload
%autoreload 2

In [362]:
import json
import sys
import re
import os
from pathlib import Path
from functools import partial


import blpapi
import clarion
import ipywidgets as w
import ipyvuetify as v
import numpy as np
import pandas as pd
import QuantLib as ql
from IPython.display import display
from ipydatagrid import DataGrid, TextRenderer, BarRenderer, Expr
from bqplot import LinearScale, ColorScale, OrdinalColorScale, OrdinalScale
from clarion import positions

from api.gui.base import View
from api.blpw import BlpQuery
from api.data.base import data_path
from api.data.curves import curves
from api.data.utils import add_pcs, parse_tenor
from api.pricing.cds import get_isda_cds
from api.data.processing import apply_function
from api.pricing.utils import (
    get_cds_specs,
    get_cds_curve,
    get_cds_spreads,
    get_risk_free_curve,
    get_cds_implied_price,
    get_curve,
)

In [5]:
env = {
    'CLARION_API_USER': 'giovambattista.perciaccante@brevanhoward.com',
    'CLARION_API_PASS': 'Macrobot123!',
    'USE_PYTHON_SDK_SSO': 'True',
    'http_proxy': 'http://proxyldn.rivagecapital.com:8080',
    'https_proxy': 'http://proxyldn.rivagecapital.com:8080',
}
for k, v in env.items():
    os.environ[k] = v

In [ ]:
os.enviro

#### Settings

In [49]:
dt_fmt = '%Y-%m-%d'
bbg_dt_fmt = '%Y%m%d'

#### Bloomberg Connection

In [50]:
bq = BlpQuery(timeout=50000).start()
self = bq

In [339]:
dl_btn = v.Btn(
    fab=True,
    # dark=True,
    small=True,
    plain=True,
    class_='mx-0',
    children=[v.Icon(children=['mdi-download'])]
)

refresh_btn = v.Btn(
    fab=True,
    # dark=True,
    small=True,
    plain=True,
    class_='mx-2',
    children=[v.Icon(children=['mdi-sync'])]
)

pcs_tf = v.TextField(
    v_model=None,
    dense=True,
    label='pcs',
    clearable=True
)

In [360]:
class Comp(View):
    
    def make_widgets(self):
        paste_btn = v.Btn(
            fab=True,
            # dark=True,
            small=True,
            plain=True,
            class_='mx-0',
            children=[v.Icon(children=['mdi-content-paste'])]
        )

        refresh_btn = v.Btn(
            fab=True,
            # dark=True,
            small=True,
            plain=True,
            class_='mx-2',
            children=[v.Icon(children=['mdi-sync'])]
        )

        pcs_tf = v.TextField(
            v_model='BGN',
            outlined=True,
            dense=True,
            label='pcs',
            clearable=True
        )
        self.paste_btn = dl_btn
        self.refresh_btn = refresh_btn
        self.pcs_tf = pcs_tf
        
        self.out = w.Output()
    
    def make_view(self):
        
        paste_btn = self.paste_btn
        refresh_btn = self.refresh_btn
        pcs_tf = self.pcs_tf
        out = self.out
        
        param_box = v.Container(
            children=[
                v.Row(
                    children=[
                        v.Col(
                            cols=2,
                            children=[pcs_tf],
                            class_="my-0 py-0"
                        ),
                        v.Col(
                            cols=2,
                            children=[paste_btn, refresh_btn],
                            class_="my-0 py-0"
                        ),

                    ],
                    align_content='center',
                ),
            ]
        )
        
        view = w.VBox(
            children=[
                param_box,
                out,
            ]
        )

        self.param_box = param_box
        self.view = view
    
    def link(self):
        pass

In [361]:
self = Comp()
self

In [359]:
def on_click_paste(widget, event, data, self):
    data = pd.read_clipboard(sep='\t', header=[1])
    data['Notional'] = data['Notional'].str.replace(',', '').astype(float)
    cols = [
        'Issuer', 
        'Trade Type',
        'Description',
        'Ccy',
        'Expiry/ Maturity',
        'ISIN',
        'Total',
        'Notional',
        'Quoted Price',
    ]
    data_ = data[cols].dropna(subset='ISIN')
    data_ = data_.drop('Notional', axis=1)\
        .drop_duplicates(subset='ISIN')\
        .merge(data_.groupby('ISIN', as_index=False, sort=False)['Notional'].sum(), on='ISIN')
    data_ = data_[[*data_.columns[:-2], data_.columns[-1], data_.columns[-2]]]
    data_ = data_.loc[data_['Notional'] != 0, :]
    
    self.data = data
    self.data_ = data_

In [370]:
out = self.out

In [363]:
self.paste_btn.on_event(
    'click', 
    partial(on_click_paste, self=self)
)

In [371]:
self.refresh_btn.on_event(
    'click', 
    partial(
        out.capture(on_click_refresh_grid), 
        self=self
    )
)

In [ ]:
data = pd.read_clipboard(sep='\t', header=[1])
data['Notional'] = data['Notional'].str.replace(',', '').astype(float)

In [88]:
cols = [
    'Issuer', 
    'Trade Type',
    'Description',
    'Ccy',
    'Expiry/ Maturity',
    'ISIN',
    'Total',
    'Notional',
    'Quoted Price',
]

In [155]:
data_ = data[cols].dropna(subset='ISIN')
data_ = data_.drop('Notional', axis=1)\
    .drop_duplicates(subset='ISIN')\
    .merge(data_.groupby('ISIN', as_index=False, sort=False)['Notional'].sum(), on='ISIN')
data_ = data_[[*data_.columns[:-2], data_.columns[-1], data_.columns[-2]]]
data_ = data_.loc[data_['Notional'] != 0, :]

In [314]:
def get_securities(securities, id_type='isin', pcs=None, sep=None):
    if pcs:
        if id_type.lower() == 'isin':
            sep = '@'
        else: 
            sep = ' '
        securities = add_pcs(securities, source=pcs, sep=sep)
    
    return [f'/{id_type}/{sec}' for sec in securities]

In [315]:
pcs = 'BGN'
sep = '@'

In [316]:
securities = get_securities(data_['ISIN'], pcs=pcs)

In [368]:
def bbg_comp(data_, pcs=None):
    securities = get_securities(data_['ISIN'], pcs=pcs)
    fields = [
        'px_last',
    ]
    
    bbg_data = bq.bdp(
        securities=securities,
        fields=fields
    )
    bbg_data['security'] = bbg_data['security'].replace(data_['ISIN'].set_axis(securities))
    comp_data = data_.merge(bbg_data, how='left', left_on='ISIN', right_on='security').drop('security', axis=1)
    comp_data['price_diff'] = (comp_data['Quoted Price'] - comp_data['px_last']).abs()
    comp_data['notional_diff'] = (comp_data['price_diff'] * comp_data['Notional']).abs()
    comp_data = comp_data.sort_values('price_diff', ascending=False)
    
    return comp_data

In [369]:
def on_click_refresh_grid(widget, event, data, self):
    data_ = self.data_
    pcs = self.pcs_tf.v_model
    comp_data = bbg_comp(data_, pcs)
    self.comp_data = comp_data
    dg = make_datagrid(comp_data)
    self.dg = dg
    display(dg)

In [317]:
fields = [
    'px_last',
]

In [318]:
bbg_data = bq.bdp(
    securities=securities,
    fields=fields
)

In [319]:
bbg_data['security'] = bbg_data['security'].replace(data_['ISIN'].set_axis(securities))

In [320]:
comp_data = data_.merge(bbg_data, how='left', left_on='ISIN', right_on='security').drop('security', axis=1)

In [321]:
comp_data['price_diff'] = (comp_data['Quoted Price'] - comp_data['px_last']).abs()
comp_data['notional_diff'] = (comp_data['price_diff'] * comp_data['Notional']).abs()
comp_data = comp_data.sort_values('price_diff', ascending=False)

In [322]:
def millify(cell):
    n = abs(float(cell.value))
    
    if n < 1000:
        return format(cell.value, '.3f') 
    elif n < 1000000:
        return format(cell.value / 1000, '.3f') + 'k'
    elif n < 1000000000:
        return format(cell.value / 1000000, '.3f') + 'm'
    elif n < 1000000000000:
        return format(cell.value / 1000000000, '.3f') + 'b'
    else:
        return format(cell.value / 1000000000000, '.3f') + 't'
    
def text_color(cell):
    pass

In [366]:
def make_datagrid(comp_data):
    renderers = {

        'Notional': TextRenderer(
            text_value=Expr(millify),
            text_color=Expr('"red" if cell.value < 0 else default_value')
        ),
        'notional_diff': BarRenderer(
            text_value=Expr(millify),
            horizontal_alignment="center",
            bar_color=ColorScale(min=comp_data['notional_diff'].min(), max=comp_data['notional_diff'].max()),
            bar_value=LinearScale(min=comp_data['notional_diff'].min(), max=comp_data['notional_diff'].max())
        ),
        'price_diff': BarRenderer(
            text_value=Expr("format(cell.value, '.2f')"),
            horizontal_alignment="center",
            bar_color=ColorScale(min=comp_data['price_diff'].min(), max=comp_data['price_diff'].max()),
            bar_value=LinearScale(min=comp_data['price_diff'].min(), max=comp_data['price_diff'].max())
        )
    }
    
    dg = DataGrid(comp_data, renderers=renderers)
    dg.auto_fit_columns = True
    return dg


In [331]:
renderers = {

    'Notional': TextRenderer(
        text_value=Expr(millify),
        text_color=Expr('"red" if cell.value < 0 else default_value')
    ),
    'notional_diff': BarRenderer(
        text_value=Expr(millify),
        horizontal_alignment="center",
        bar_color=ColorScale(min=comp_data['notional_diff'].min(), max=comp_data['notional_diff'].max()),
        bar_value=LinearScale(min=comp_data['notional_diff'].min(), max=comp_data['notional_diff'].max())
    ),
    'price_diff': BarRenderer(
        text_value=Expr("format(cell.value, '.2f')"),
        horizontal_alignment="center",
        bar_color=ColorScale(min=comp_data['price_diff'].min(), max=comp_data['price_diff'].max()),
        bar_value=LinearScale(min=comp_data['price_diff'].min(), max=comp_data['price_diff'].max())
    )
}

In [332]:
dg = DataGrid(comp_data, renderers=renderers)
dg.auto_fit_columns = True

In [333]:
dg

DataGrid(auto_fit_columns=True, auto_fit_params={'area': 'all', 'padding': 30, 'numCols': None}, corner_render…